In [1]:
import numpy as np
from itertools import compress

In [2]:
crps = np.load("results/active_learning/results/crps.npy")
log = np.load("results/active_learning/results/log.npy")
var = np.load("results/active_learning/results/var.npy")
kernel = np.load("results/active_learning/results/kernel.npy")

In [3]:
log

array([[[2.84569763, 3.43028098],
        [1.57967112, 1.1075691 ],
        [1.17126024, 0.93329428]]])

In [4]:
crps

array([[[2.83205803, 3.35903487],
        [1.27422799, 1.36557783],
        [1.08067279, 0.7157672 ]]])

In [5]:
var

array([[[2.94700984, 3.33554418],
        [1.26057446, 1.3707563 ],
        [1.06670298, 0.71299443]]])

In [6]:
kernel

array([[[3.0245373 , 3.17109397],
        [1.54877279, 1.021765  ],
        [1.09570829, 0.67720995]]])

In [2]:
test = np.array([5,2,200,16])
np.argsort(test)

array([1, 0, 3, 2])

In [29]:
train_list = [1,2,3,4,5, 6]*20
n_samples = len(train_list)
perm = np.random.permutation(n_samples)
n_samples

120

In [41]:
# Get boolean lists
train_index = perm[0:10]
train_bool_index = np.isin(perm, train_index).tolist()
pool_bool_index = (~np.array(train_bool_index)).tolist()
pool_index = perm[pool_bool_index]

In [47]:
len(train_index)

10

In [48]:
len(pool_index)

110

In [38]:
# Update train list
train = list(compress(train_list, train_bool_index))
pool = list(compress(train_list, pool_bool_index))

In [52]:
new_indices = np.array([7,13,26])

In [55]:
train_index = np.concatenate([train_index, new_indices])

In [56]:
train_bool_index = np.isin(perm, train_index).tolist()
pool_bool_index = (~np.array(train_bool_index)).tolist()
pool_index = perm[pool_bool_index]

In [60]:
train_index

array([111,  93,  94,  71,  88,  89,  92, 119,  96,   6,   7,  13,  26])

In [61]:
pool_index.sort()
pool_index

array([  0,   1,   2,   3,   4,   5,   8,   9,  10,  11,  12,  14,  15,
        16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,
        69,  70,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  90,  91,  95,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115,
       116, 117, 118])